# Cassandra Vector Store

[Apache Cassandra®](https://cassandra.apache.org) is a NoSQL, row-oriented, highly scalable and highly available database.
Newest Cassandra releases natively [support](https://cwiki.apache.org/confluence/display/CASSANDRA/CEP-30%3A+Approximate+Nearest+Neighbor(ANN)+Vector+Search+via+Storage-Attached+Indexes) Vector Similarity Search.

**This notebook shows the basic usage of Cassandra as a Vector Store in LlamaIndex.**

To run this notebook you need either a running Cassandra cluster equipped with Vector Search capabilities (in pre-release at the time of writing) or a DataStax Astra DB instance running in the cloud (you can get one for free at [datastax.com](https://astra.datastax.com)). _This notebook covers both choices._ Check [cassio.org](https://cassio.org/start_here/) for more information, quickstarts and tutorials.

## Setup

In [ ]:
!pip install "cassio>=0.0.7"

In [2]:
import os

from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Document,
    StorageContext,
)
from llama_index.vector_stores import CassandraVectorStore

### Please provide database connection parameters and secrets

First you need a database connection (a `cassandra.cluster.Session` object).

Make sure you have either a vector-capable running Cassandra cluster or an [Astra DB](https://astra.datastax.com) instance in the cloud.

In [3]:
import os
import getpass

database_mode = (input("\n(C)assandra or (A)stra DB? ")).upper()

keyspace_name = input("\nKeyspace name? ")

if database_mode == "A":
    ASTRA_DB_APPLICATION_TOKEN = getpass.getpass('\nAstra DB Token ("AstraCS:...") ')
    #
    ASTRA_DB_SECURE_BUNDLE_PATH = input("Full path to your Secure Connect Bundle? ")
elif database_mode == "C":
    CASSANDRA_CONTACT_POINTS = input(
        "Contact points? (comma-separated, empty for localhost) "
    ).strip()


(C)assandra or (A)stra DB? A

Keyspace name? my_cassandra_keyspace

Astra DB Token ("AstraCS:...") ········
Full path to your Secure Connect Bundle? /home/USER/secure-bundle.zip


In [4]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

if database_mode == "C":
    if CASSANDRA_CONTACT_POINTS:
        cluster = Cluster(
            [cp.strip() for cp in CASSANDRA_CONTACT_POINTS.split(",") if cp.strip()]
        )
    else:
        cluster = Cluster()
    session = cluster.connect()
elif database_mode == "A":
    ASTRA_DB_CLIENT_ID = "token"
    cluster = Cluster(
        cloud={
            "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
        },
        auth_provider=PlainTextAuthProvider(
            ASTRA_DB_CLIENT_ID,
            ASTRA_DB_APPLICATION_TOKEN,
        ),
    )
    session = cluster.connect()
else:
    raise NotImplementedError

### Please provide OpenAI access key

In order use embeddings by OpenAI you need to supply an OpenAI API Key:

In [5]:
import openai

OPENAI_API_KEY = getpass.getpass("OpenAI API Key:")
openai.api_key = OPENAI_API_KEY

OpenAI API Key:········


## Creating and populating the Vector Store

You will now load some essays by Paul Graham from a local file and store them into the Cassandra Vector Store.

In [6]:
# load documents
documents = SimpleDirectoryReader("../data/paul_graham").load_data()
print(f"Total documents: {len(documents)}")
print(f"First document, id: {documents[0].doc_id}")
print(f"First document, hash: {documents[0].hash}")
print(
    f"First document, text ({len(documents[0].text)} characters):\n{'='*20}\n{documents[0].text[:360]} ..."
)

Total documents: 1
First document, id: 43ed31eb-52cf-4d19-bf33-9c312a03b833
First document, hash: 4c702b4df575421e1d1af4b1fd50511b226e0c9863dbfffeccb8b689b8448f35
First document, text (75019 characters):
		

What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined  ...


### Initialize the Cassandra Vector Store

Creation of the vector store entails creation of the underlying database table if it does not exist yet:

In [7]:
cassandra_store = CassandraVectorStore(
    session=session,
    keyspace=keyspace_name,
    table="cassandra_vector_table_1",
    embedding_dimension=1536,
)

Now wrap this store into an `index` LlamaIndex abstraction for later querying:

In [8]:
storage_context = StorageContext.from_defaults(vector_store=cassandra_store)

index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

Note that the above `from_documents` call does several things at once: it splits the input documents into chunks of manageable size, computes embedding vectors for each chunk, and stores them all in the Cassandra Vector Store.

## Querying the store

### Basic querying

In [9]:
query_engine = index.as_query_engine()
response = query_engine.query("Why did the author choose to work on AI?")
print(response.response)


The author chose to work on AI because of his fascination with the novel The Moon is a Harsh Mistress, which featured an intelligent computer called Mike, and a PBS documentary that showed Terry Winograd using SHRDLU. He was also drawn to the idea of AI because it seemed like a powerful field of study, and he was eager to explore the potential of computers and programming.


### MMR-based queries

The MMR (maximal marginal relevance) method is designed to fetch text chunks from the store that are at the same time relevant to the query but as different as possible from each other, with the goal of providing a broader context to the building of the final answer:

In [10]:
query_engine = index.as_query_engine(vector_store_query_mode="mmr")
response = query_engine.query("Why did the author choose to work on AI?")
print(response.response)


The author chose to work on AI because he was impressed and envious of his friend who had built a computer kit and was able to type programs into it. He was also inspired by a novel by Heinlein called The Moon is a Harsh Mistress, which featured an intelligent computer called Mike, and a PBS documentary that showed Terry Winograd using SHRDLU. He was also disappointed with philosophy courses in college, which he found to be boring, and he wanted to work on something that seemed more powerful.


## Connecting to an existing store

Since this store is backed by Cassandra, it persistent by definition. So, if you want to connect to a store that was already created and populated, here is how:

In [11]:
new_store_instance = CassandraVectorStore(
    session=session,
    keyspace=keyspace_name,
    table="cassandra_vector_table_1",
    embedding_dimension=1536,
)

# Create index (from preexisting stored vectors)
new_index_instance = VectorStoreIndex.from_vector_store(vector_store=new_store_instance)

# now you can do querying, etc:
query_engine = index.as_query_engine(similarity_top_k=5)
response = query_engine.query("What did the author study prior to working on AI?")

In [12]:
print(response.response)



The author studied philosophy and painting, worked on spam filters, and wrote essays prior to working on AI.


## Removing documents from the index

First get an explicit list of documents, or "nodes", from a `Retriever` spawned from the index:

In [13]:
retriever = new_index_instance.as_retriever(
    vector_store_query_mode="mmr",
    similarity_top_k=3,
    vector_store_kwargs={"mmr_prefetch_factor": 4},
)
nodes_with_scores = retriever.retrieve(
    "What did the author study prior to working on AI?"
)

In [14]:
print(f"Found {len(nodes_with_scores)} nodes.")
for idx, node_with_score in enumerate(nodes_with_scores):
    print(f"    [{idx}] score = {node_with_score.score}")
    print(f"        id    = {node_with_score.node.node_id}")
    print(f"        text  = {node_with_score.node.text[:100]} ...")

Found 3 nodes.
    [0] score = 0.42589144520149874
        id    = ed674b40-f7fc-41d4-b419-1efbf81ae279
        text  = What I Worked On

February 2021

Before college the two main things I worked on, outside of school,  ...
    [1] score = -0.0012061281453193962
        id    = 367dc1fc-2996-4eb9-9b5e-8ecde7f7ab4a
        text  = been explored. But all I wanted was to get out of grad school, and my rapidly written dissertation s ...
    [2] score = 0.025454533089838027
        id    = 2faf35d8-fd91-4b3b-ba04-505ae54fd9e2
        text  = showed Terry Winograd using SHRDLU. I haven't tried rereading The Moon is a Harsh Mistress, so I don ...


Delete the first node from the index:

In [15]:
new_store_instance.delete(nodes_with_scores[0].node.node_id)

Repeat the very same query and check the results now:

In [16]:
nodes_with_scores = retriever.retrieve(
    "What did the author study prior to working on AI?"
)

print(f"Found {len(nodes_with_scores)} nodes.")
for idx, node_with_score in enumerate(nodes_with_scores):
    print(f"    [{idx}] score = {node_with_score.score}")
    print(f"        id    = {node_with_score.node.node_id}")
    print(f"        text  = {node_with_score.node.text[:100]} ...")

Found 3 nodes.
    [0] score = 0.4103668740067556
        id    = 2faf35d8-fd91-4b3b-ba04-505ae54fd9e2
        text  = showed Terry Winograd using SHRDLU. I haven't tried rereading The Moon is a Harsh Mistress, so I don ...
    [1] score = 0.0036428220067193373
        id    = 367dc1fc-2996-4eb9-9b5e-8ecde7f7ab4a
        text  = been explored. But all I wanted was to get out of grad school, and my rapidly written dissertation s ...
    [2] score = 0.02135211338932269
        id    = 27752a42-b37e-4933-9b4b-7d646c8b0b45
        text  = 1960 paper.

But if so there's no reason to suppose that this is the limit of the language that migh ...
